In [71]:
import numpy as np
import cv2
import os

from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import RMSprop, Adam
from sklearn.externals import joblib
from sklearn import cluster

import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.Session(config=config)

# 設定 Keras 使用的 TensorFlow Session
KTF.set_session(sess)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

animeface_cascade        = cv2.CascadeClassifier('xml/lbpcascade_animeface.xml')
frontalface_cascade      = cv2.CascadeClassifier('xml/haarcascade_frontalface_default.xml')
head_and_shouler_cascade = cv2.CascadeClassifier('xml/HS.xml')


INPUT_SIZE   = 64
INPUT_SIZE_2 = int(INPUT_SIZE / 2)
INPUT_SIZE_8 = int(INPUT_SIZE / 8)
OUTPUT_SIZE  = 1

DETECT_VAR_1 = [[1.1, 1, 0,  0], [1.1, 1, 20, 5]] # add train data
DETECT_VAR_2 = [[1.2, 3, 0,  0], [1.2, 3, 20, 5]]

LABEL_NUM = 7
CROPS = ['NO', 'UL', 'UR', 'DL', 'DR', 'MID', 'EDGE']
PLAN  = '2DCNN'

def My_Crop (IMG, C):
    
    if   (C == CROPS[0]): # NO
        return IMG
    elif (C == CROPS[1]): # UL
        return cv2.resize(IMG[0:INPUT_SIZE_2, 0:INPUT_SIZE_2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[2]): # UR
        return cv2.resize(IMG[0:INPUT_SIZE_2, INPUT_SIZE_2:INPUT_SIZE_2*2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[3]): # DL
        return cv2.resize(IMG[INPUT_SIZE_2:INPUT_SIZE_2*2, 0:INPUT_SIZE_2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[4]): # DR
        return cv2.resize(IMG[INPUT_SIZE_2:INPUT_SIZE_2*2, INPUT_SIZE_2:INPUT_SIZE_2*2], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[5]): # MID
        return cv2.resize(IMG[INPUT_SIZE_8:INPUT_SIZE_2+INPUT_SIZE_8*3, INPUT_SIZE_8:INPUT_SIZE_2+INPUT_SIZE_8*3], (INPUT_SIZE, INPUT_SIZE))
    elif (C == CROPS[6]): # EDGE
        """
        temp = cv2.resize(IMG, (INPUT_SIZE*2, INPUT_SIZE*2))
        temp = cDETECT_VARv2.Canny(temp, 200, 255)
        temp = cv2.dilate(temp, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations=1)
        temp = cv2.erode (temp, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2)), iterations=1)
        temp = cv2.resize(temp, (INPUT_SIZE, INPUT_SIZE))
        return np.repeat(temp, 3)
        """
        return np.repeat(cv2.Canny(IMG, 200, 255), 3)
    else:
        return np.zeros(INPUT_SIZE * INPUT_SIZE * 3)

# 動漫人臉偵測 by lbpcascade_animeface.xml

In [72]:
# input PATH or IMAGE
# A and B (cascade) , dW (scale) , dH (move up and down)
def Get_Image_Animeface_Detection (IP, IMG, A, B, dW, dH):
    
    if(IP != None):
        img  = cv2.imread(IP)
    else:
        img = IMG
        
    if img is None:
        return [img, []]
    
    #print(img.shape)    
    temp = 256 / img.shape[0]
    gray = cv2.cvtColor(cv2.resize(img, (int(img.shape[1] * temp), int(img.shape[0] * temp))), cv2.COLOR_BGR2GRAY)
    
    #img = cv2.resize(img, (int(img.shape[1] * temp), int(img.shape[0] * temp)))
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    """
    list_faces = []
    
    faces1 = animeface_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces1) > 0):
        list_faces.append(faces1.tolist())
    
    faces2 = frontalface_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces2) > 0):
        list_faces.append(faces2.tolist())
        
    faces3 = head_and_shouler_cascade.detectMultiScale(gray, 1.1, 1)
    if(len(faces3) > 0):
        list_faces.append(faces3.tolist())
    
    faces = np.array(eval('[%s]'%repr(list_faces).replace('[', '').replace(']', ''))).reshape(-1, 4)
    """
    #print(gray.shape)
    faces = animeface_cascade.detectMultiScale(gray, A, B)
    if(len(faces) > 0):
        faces = (faces / temp).astype(np.int32)
    
    # scale and move window
    faces_ = []
    for (x,y,w,h) in faces:
        
        if (dW > 0):
            w_ = int(w / dW)
            x -= w_
            w += (w_ * 2)
        
        if (dH > 0):
            y -= int(h / dH)
        h = w
        
        if(x >= 0 and y >= 0 and x+w <= img.shape[1] and y+h <= img.shape[0]):
            faces_.append([x,y,w,h])
    
    return [img, faces_]

In [75]:
dIMAGE_PATH = 'test3.jpg'

faces = []
for (a,b,dw,dh) in DETECT_VAR_2:
    img, faces_ = Get_Image_Animeface_Detection (dIMAGE_PATH, None, a, b, dw, dh)
    faces += faces_

if img is not None:
    for (x,y,w,h) in faces:
        #print(w,h)
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)

    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

else:
    print('Image is None')

# 動畫人臉擷取 by lbpcascade_animeface.xml

In [76]:
D_FRAME  = 30  # N楨取一次
MIN_EDGE = 200 # 
# NAME = EPISODE_NAME
# output = EPISODE_dW-dH_ID_NAME.jpg
# ID -> EPISODE_dW-dH
def Crop_Video_Animeface_Detection (IP, OP, NAME, DV):
    
    EP, NAME = NAME.split('_', 1)

    # 開啟影片檔案
    cap = cv2.VideoCapture(IP)

    if not os.path.exists(OP):
        os.makedirs(OP)

    # 以迴圈從影片檔案讀取影格，並顯示出來
    image_i = np.zeros(len(DV)).astype(np.uint32)
    frame_i = 0
    while(cap.isOpened()):
    
        ret, img = cap.read()
        
        if img is None:
            break

        frame_i = frame_i + 1
        if(frame_i % D_FRAME != 0):
            continue
        
        var_i = 0
        for (a,b,dw,dh) in DV:
            
            img, faces = Get_Image_Animeface_Detection (None, img, a, b, dw, dh)

            # 取出所有偵測的結果
            for (x, y, w, h) in faces:

                if(w < MIN_EDGE):
                    continue

                face = cv2.resize(img[y:y+h, x:x+w], (256, 256))
                cv2.imwrite(OP + '/' + EP + '_' + str(dw) + '-' + str(dh) + '_' + str(image_i[var_i]) + '_' + NAME + '.jpg', face)
                image_i[var_i] = image_i[var_i] + 1
            var_i = var_i + 1

    cap.release()

In [52]:
VIDEO_PATH = 'movie/Oreimo/'
VIDEO_NAME = '_Oreimo'
VIDEO_TYPE = '.mp4'

for i in range(1,8):
    VIDEO_NAME = str(i) + '_Oreimo'
    Crop_Video_Animeface_Detection (VIDEO_PATH+VIDEO_NAME+VIDEO_TYPE, VIDEO_NAME, VIDEO_NAME, DETECT_VAR_1)

# 篩選訓練資料 by k-means

In [77]:
def getFirst(elem):
    return elem[0]

def Choose_Data_by_Kmeans (IP, OP, N):
    
    for dCHARACTER_DIR_NAME in os.listdir(IP):
    
        label, name = dCHARACTER_DIR_NAME.split('_', 1)
        list_imgs   = []
    
        IP_d = IP + '/' + dCHARACTER_DIR_NAME
        for dIMAGE in os.listdir(IP_d):
            
            img = cv2.resize(cv2.imread(IP_d + '/' + dIMAGE), (INPUT_SIZE, INPUT_SIZE)).ravel()
            list_imgs.append(img)
    
        imgs = np.array(list_imgs).astype(np.float32) / 255
        print(dCHARACTER_DIR_NAME, imgs.shape)
        #print('imgs :\n', imgs)
    
        if (imgs.shape[0] >= N):
            
            if not os.path.exists(OP):
                os.makedirs(OP)
            
            KMEANS_MODEL = cluster.KMeans(n_clusters = N, random_state = 42)
            imgs_kmeans = KMEANS_MODEL.fit(imgs)
            #print('imgs_kmeans :\n', imgs_kmeans.labels_)

            labs_imgs = list(zip(imgs_kmeans.labels_, imgs))
            labs_imgs.sort(key=getFirst)
            #print('labs_imgs :\n', labs_imgs)

            choose_i = 0
            for ele in labs_imgs:
                
                if (ele[0] == choose_i):

                    img_ = (np.array(ele[1]) * 255).reshape(INPUT_SIZE, INPUT_SIZE, 3).astype(np.int32)

                    OP_d = OP + '/' + dCHARACTER_DIR_NAME
                    if not os.path.exists(OP_d):
                        os.makedirs(OP_d)
                    cv2.imwrite(OP_d + '/' + str(choose_i) + '.jpg', img_)

                    choose_i += 1

In [78]:
dIMAGE_DIR_PATH = 'data'
dTRAIN_DIR_PATH = 'data_train'
IMAGE_OUTNUM = 120

Choose_Data_by_Kmeans (dIMAGE_DIR_PATH, dTRAIN_DIR_PATH, IMAGE_OUTNUM)

0_Izumi Sagiri (799, 12288)
1_Izumi Masamune (1682, 12288)
2_Yamada Erufu (591, 12288)
3_Jinno Megumi (337, 12288)
4_Muramasa Senju (245, 12288)
5_Takasago Tomoe (123, 12288)
6_Kagurazaka Ayame (173, 12288)


# 建立模型 by CNN

In [79]:
def Build_Model (IP, C, P, TRAIN_N, TEST_N):

    list_labels        = []
    list_train_numbers = []
    list_test_numbers  = []

    list_train_images  = []
    list_test_images   = []

    for dCHARACTER_DIR_NAME in os.listdir(IP):
    
        label, name = dCHARACTER_DIR_NAME.split('_', 1)

        list_labels.append(int(label))
        list_train_numbers.append(TRAIN_N)
        list_test_numbers.append(TEST_N)

        i = 0
        IP_d = IP + '/' + dCHARACTER_DIR_NAME
        for dIMAGE in os.listdir(IP_d):
            
            img_rgb = cv2.imread(IP_d + '/' + dIMAGE)
            img = My_Crop(img_rgb, C)
        
            if(i < TRAIN_NUM):
                list_train_images.append(img.tolist())
            else:
                list_test_images.append(img.tolist())
            i = i + 1
        
    labels        = np.array(list_labels       ).reshape(-1)
    train_numbers = np.array(list_train_numbers).reshape(-1)
    test_numbers  = np.array(list_test_numbers ).reshape(-1)
    
    if   (P == '1DCNN'):
        train_images = np.array(list_train_images).reshape(-1, INPUT_SIZE*INPUT_SIZE*3).astype(np.float32) / 255
        test_images  = np.array(list_test_images ).reshape(-1, INPUT_SIZE*INPUT_SIZE*3).astype(np.float32) / 255

        train_images = np.expand_dims(train_images, axis=2)
        test_images  = np.expand_dims(test_images,  axis=2)

    elif (P == '2DCNN'):
        train_images = np.array(list_train_images).reshape(-1, INPUT_SIZE, INPUT_SIZE*3).astype(np.float32) / 255
        test_images  = np.array(list_test_images ).reshape(-1, INPUT_SIZE, INPUT_SIZE*3).astype(np.float32) / 255

        train_images = np.expand_dims(train_images, axis=3)
        test_images  = np.expand_dims(test_images,  axis=3)

    train_labels = np_utils.to_categorical(labels.repeat(train_numbers), len(labels)).astype(np.uint32)
    test_labels  = np_utils.to_categorical(labels.repeat(test_numbers),  len(labels)).astype(np.uint32)


    print("train_images.shape : ", train_images.shape)
    print("test_images.shape  : ", test_images.shape)
    print("train_labels.shape : ", train_labels.shape)


    # build model
    model = Sequential()

    if  (P == '1DCNN'):
        model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(INPUT_SIZE*INPUT_SIZE*3, 1)))
        model.add(Flatten())
        model.add(Dense(OUTPUT_SIZE))
        model.compile(loss='mse', optimizer='adam')
        train_history = model.fit(train_images, train_labels, epochs=40, batch_size=10, shuffle=False, validation_data=(test_images, test_labels), verbose=2)

    elif (P == '2DCNN'):
        
        model.add(Conv2D(filters=16, kernel_size=(3,3), padding='same', activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE*3, 1)))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(len(labels), activation='softmax'))
        rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer=rmsprop, loss='categorical_crossentropy', metrics=['accuracy'])
        #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        train_history = model.fit(train_images, train_labels, epochs=100, batch_size=80, shuffle=False, 
                                  validation_data=(test_images, test_labels), verbose=2)

    model.save(C + '_' + P + '.h5')


In [80]:
dTRAIN_DIR_PATH = 'data_train'
CROP = 'NO'
TRAIN_NUM = 110
TEST_NUM  = 10

Build_Model (dTRAIN_DIR_PATH, CROP, PLAN, TRAIN_NUM, TEST_NUM)

train_images.shape :  (770, 64, 192, 1)
test_images.shape  :  (70, 64, 192, 1)
train_labels.shape :  (770, 7)
Train on 770 samples, validate on 70 samples
Epoch 1/100
 - 2s - loss: 14.0229 - acc: 0.0416 - val_loss: 13.8155 - val_acc: 0.1429
Epoch 2/100
 - 1s - loss: 11.8016 - acc: 0.2104 - val_loss: 6.8525 - val_acc: 0.1429
Epoch 3/100
 - 1s - loss: 6.3460 - acc: 0.4000 - val_loss: 6.1146 - val_acc: 0.1429
Epoch 4/100
 - 1s - loss: 3.7335 - acc: 0.4662 - val_loss: 3.6020 - val_acc: 0.1429
Epoch 5/100
 - 1s - loss: 3.0045 - acc: 0.4416 - val_loss: 3.4989 - val_acc: 0.1429
Epoch 6/100
 - 0s - loss: 2.7225 - acc: 0.4377 - val_loss: 2.9032 - val_acc: 0.1429
Epoch 7/100
 - 1s - loss: 2.0330 - acc: 0.3961 - val_loss: 2.2918 - val_acc: 0.1857
Epoch 8/100
 - 0s - loss: 1.5357 - acc: 0.4273 - val_loss: 1.7873 - val_acc: 0.2286
Epoch 9/100
 - 0s - loss: 1.1454 - acc: 0.5026 - val_loss: 1.4975 - val_acc: 0.2714
Epoch 10/100
 - 0s - loss: 0.8939 - acc: 0.5922 - val_loss: 1.4724 - val_acc: 0.3000
E

Epoch 95/100
 - 0s - loss: 6.1215e-05 - acc: 1.0000 - val_loss: 7.9827e-04 - val_acc: 1.0000
Epoch 96/100
 - 0s - loss: 5.3880e-05 - acc: 1.0000 - val_loss: 7.0136e-04 - val_acc: 1.0000
Epoch 97/100
 - 0s - loss: 4.6377e-05 - acc: 1.0000 - val_loss: 7.8129e-04 - val_acc: 1.0000
Epoch 98/100
 - 0s - loss: 6.7311e-05 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000
Epoch 99/100
 - 0s - loss: 5.4136e-05 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 100/100
 - 0s - loss: 3.6848e-05 - acc: 1.0000 - val_loss: 0.0025 - val_acc: 1.0000


In [81]:
from keras.models import load_model

PATH = 'model/Y/'
LABEL_NUM = 7
CROP_LIST  = ['NO', 'MID', 'EDGE']
THRESHOLD_P = 0.5

MARK = 'L' + str(LABEL_NUM) + '-C' + str(len(CROP_LIST)) + '-P' + str(int(THRESHOLD_P*10)) + '-PREDICT_'

for CROP in CROP_LIST:
    
    if   (CROP == CROPS[0]):
        NN0 = load_model(PATH + CROPS[0] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[1]):
        NN1 = load_model(PATH + CROPS[1] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[2]):
        NN2 = load_model(PATH + CROPS[2] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[3]):
        NN3 = load_model(PATH + CROPS[3] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[4]):
        NN4 = load_model(PATH + CROPS[4] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[5]):
        NN5 = load_model(PATH + CROPS[5] + '_' + PLAN + '.h5')
    elif (CROP == CROPS[6]):
        NN6 = load_model(PATH + CROPS[6] + '_' + PLAN + '.h5')

# 動畫人臉分類 (圖片)

In [60]:
# IMG = INPUT_SIZE * INPUT_SIZE
def Get_Image_Animeface_Recognition (IMG, CS, P):
    
    vote = np.zeros(LABEL_NUM)
    
    for CROP in CS:
        
        IMG_ = My_Crop(IMG, CROP)
        
        if   (P == '1DCNN'):
            test_image = np.array(IMG_).reshape(1,INPUT_SIZE * INPUT_SIZE * 3).astype(np.float32) / 255
            test_image = np.expand_dims(test_image, axis=2)
        elif (P == '2DCNN'):
            test_image = np.array(IMG_).reshape(1,INPUT_SIZE, INPUT_SIZE * 3).astype(np.float32) / 255
            test_image = np.expand_dims(test_image, axis=3)
        
        if   (CROP == CROPS[0]):
            list_predicts =  NN0.predict(test_image)[0]
        elif (CROP == CROPS[1]):
            list_predicts =  NN1.predict(test_image)[0]
        elif (CROP == CROPS[2]):
            list_predicts =  NN2.predict(test_image)[0]
        elif (CROP == CROPS[3]):
            list_predicts =  NN3.predict(test_image)[0]
        elif (CROP == CROPS[4]):
            list_predicts =  NN4.predict(test_image)[0]
        elif (CROP == CROPS[5]):
            list_predicts =  NN5.predict(test_image)[0]
        elif (CROP == CROPS[6]):
            list_predicts =  NN6.predict(test_image)[0]
            
        print(CROP, ' : ', list_predicts)
        if (np.max(list_predicts) > THRESHOLD_P):
            vote[np.argmax(list_predicts)] += 1
    
    print('VOTE : ', vote)
    
    return 'unknow' if np.max(vote) < (len(CS) / 2) else str(np.argmax(vote))

In [61]:
dIMAGE_PATH = 'test3.jpg'

img, faces = Get_Image_Animeface_Detection (dIMAGE_PATH, None, 1.1, 1, 0, 0)

image_i = 0
for (x,y,w,h) in faces:
    print(str(image_i))
    image_i += 1
    
    print('ans : ', Get_Image_Animeface_Recognition (cv2.resize(img[y:y+h, x:x+w], (INPUT_SIZE,INPUT_SIZE)), CROP_LIST, PLAN))
    print('X : ', x)
    print('Y : ', y)

0
NO  :  [5.3251100e-08 2.4673716e-07 1.4402266e-09 1.7091308e-07 9.9527884e-01
 4.7180457e-03 2.6680750e-06]
MID  :  [1.9801913e-10 1.6490128e-08 2.3419912e-11 4.2743000e-09 9.7906369e-01
 2.0929269e-02 7.0397646e-06]
EDGE  :  [6.32604817e-04 3.67770758e-06 5.03729461e-05 1.04317246e-06
 7.55734801e-01 2.43471086e-01 1.06527514e-04]
VOTE :  [0. 0. 0. 0. 3. 0. 0.]
ans :  4
X :  101
Y :  470
1
NO  :  [5.66439012e-05 2.08375084e-07 1.82268843e-02 9.67752993e-01
 1.06023625e-08 4.03310141e-09 1.39632598e-02]
MID  :  [2.3744735e-03 8.1843780e-07 9.1258341e-01 8.0138907e-02 3.7576106e-10
 1.1927809e-11 4.9024178e-03]
EDGE  :  [1.4138823e-04 4.6229010e-11 9.7576547e-01 4.3554579e-10 5.5964782e-21
 8.4931113e-14 2.4093030e-02]
VOTE :  [0. 0. 2. 1. 0. 0. 0.]
ans :  2
X :  205
Y :  396
2
NO  :  [5.1849956e-06 2.9150484e-11 9.9999344e-01 1.2081297e-06 5.1964422e-13
 3.8304031e-13 2.1336352e-07]
MID  :  [1.7381927e-07 2.6449465e-10 9.9999964e-01 4.6251651e-09 1.5177846e-14
 8.7866334e-17 2.899100

# 動畫人臉分類 (影片)

In [82]:
D_FRAME  = 60  # N楨取一次
MIN_EDGE = 200

def Video_Animeface_Recognition (IP, OP, CS, P, DV):
    
    # 開啟影片檔案
    cap = cv2.VideoCapture(IP)
    
    # 以迴圈從影片檔案讀取影格，並顯示出來
    image_i = 0
    frame_i = 0
    while(cap.isOpened()):
        
        ret, img = cap.read()
        
        if img is None:
            break
        
        frame_i = frame_i + 1
        if(frame_i % D_FRAME != 0):
            continue
        
        faces = []
        for (a,b,dw,dh) in DV:
            img, faces_ = Get_Image_Animeface_Detection (None, img, a, b, dw, dh)
            faces = faces + faces_
        
        for (x, y, w, h) in faces:
            
            if(w < MIN_EDGE):
                continue
            
            print(str(image_i))
            
            img_face = img[y:y+h, x:x+w]
            img_rgb  = cv2.resize(img_face, (INPUT_SIZE, INPUT_SIZE))
            
            ans_str = Get_Image_Animeface_Recognition (img_rgb, CS, P)
    
            if not os.path.exists(OP):
                os.makedirs(OP)

            if not os.path.exists(OP + '/' + ans_str):
                os.makedirs(OP + '/' + ans_str)

            cv2.imwrite(OP + '/' + ans_str + '/' + str(image_i) + '.jpg', img_face)
            image_i += 1

    cap.release()

In [83]:
PATH = 'movie/EroMangaSensei/'
VIDEO_NAME  = '12_EroMangaSensei'
#VIDEO_NAME  = '1_EroMangaSensei_Coser'
VIDEO_TYPE  = '.mp4'

Video_Animeface_Recognition (PATH + VIDEO_NAME + VIDEO_TYPE, MARK + VIDEO_NAME, CROP_LIST, PLAN, DETECT_VAR_2)

0
NO  :  [9.99817073e-01 5.98881206e-06 1.00950965e-05 8.44488227e-07
 1.83195780e-07 3.15170473e-06 1.62650264e-04]
MID  :  [9.9998057e-01 4.3442324e-07 3.0581662e-06 5.8669175e-10 4.8547344e-10
 9.0269602e-08 1.5768965e-05]
EDGE  :  [4.1072924e-02 3.3312272e-06 1.0786360e-02 9.4803637e-01 1.2852135e-07
 2.4804342e-07 1.0066513e-04]
VOTE :  [2. 0. 0. 1. 0. 0. 0.]
1
NO  :  [9.9987113e-01 1.3810153e-07 7.5117014e-06 6.3933328e-08 1.6265330e-08
 6.0960508e-07 1.2045872e-04]
MID  :  [9.99999285e-01 9.85740822e-10 4.63949050e-07 1.10260195e-11
 6.41606820e-12 3.78309134e-10 1.82600658e-07]
EDGE  :  [1.4794816e-05 1.6167543e-03 2.1065066e-03 1.5796434e-02 2.8235164e-02
 1.6212442e-03 9.5060909e-01]
VOTE :  [2. 0. 0. 0. 0. 0. 1.]
2
NO  :  [9.9611521e-01 3.3641867e-05 6.0353683e-05 9.6741760e-06 5.9111369e-07
 1.2197763e-05 3.7683896e-03]
MID  :  [9.99889135e-01 9.87116096e-07 7.28467376e-06 1.03191216e-08
 1.97867855e-09 1.38100972e-07 1.02543854e-04]
EDGE  :  [5.0219923e-01 2.5383905e-08 5.

23
NO  :  [9.9498820e-01 1.6202114e-06 1.1583117e-04 4.4688736e-06 6.9556307e-08
 2.2562870e-07 4.8894528e-03]
MID  :  [9.9991477e-01 1.0557013e-08 2.0747808e-05 9.3922887e-09 4.6247502e-11
 2.1101494e-10 6.4519641e-05]
EDGE  :  [1.6727037e-13 9.9111873e-01 1.1074196e-09 5.8573974e-08 2.5322205e-10
 8.5175196e-03 3.6376316e-04]
VOTE :  [2. 1. 0. 0. 0. 0. 0.]
24
NO  :  [1.1455712e-10 4.5298984e-06 3.5087977e-09 7.1339455e-05 3.3866285e-09
 1.3601719e-09 9.9992406e-01]
MID  :  [3.9606321e-10 1.1229634e-05 7.0848083e-09 8.6762353e-05 1.2285150e-07
 5.2992872e-09 9.9990189e-01]
EDGE  :  [1.7114817e-04 9.2310032e-10 3.2039409e-11 1.0452787e-11 2.0192815e-27
 3.5489674e-20 9.9982882e-01]
VOTE :  [0. 0. 0. 0. 0. 0. 3.]
25
NO  :  [4.77321438e-12 3.86535675e-08 2.59936628e-09 2.22844551e-06
 3.76603027e-10 1.17077615e-11 9.99997735e-01]
MID  :  [1.7654642e-09 5.1875185e-07 6.3861236e-08 3.0163999e-05 2.2409097e-09
 4.8508038e-09 9.9996924e-01]
EDGE  :  [2.8752071e-13 2.7799299e-13 9.2581703e-10

46
NO  :  [9.9999022e-01 4.2536880e-07 7.7565619e-07 1.4692742e-07 6.5937904e-09
 1.6899699e-07 8.1823300e-06]
MID  :  [9.9999869e-01 1.1986942e-08 4.9716633e-08 1.7595851e-10 2.3080922e-10
 3.9184581e-08 1.3018865e-06]
EDGE  :  [7.7482873e-01 2.1835746e-13 1.9099202e-06 2.2515365e-01 7.8322642e-17
 7.5435270e-14 1.5681813e-05]
VOTE :  [3. 0. 0. 0. 0. 0. 0.]
47
NO  :  [3.5021299e-09 9.9999988e-01 7.1970069e-11 3.6745664e-08 1.0382002e-08
 6.8091599e-10 8.1806839e-08]
MID  :  [3.12082484e-08 9.99997973e-01 9.88179094e-10 7.47243263e-08
 4.95184915e-09 1.46060755e-08 1.95944926e-06]
EDGE  :  [1.3359497e-07 1.8355407e-02 5.0731880e-12 3.8251734e-08 1.0098097e-07
 1.0171974e-12 9.8164427e-01]
VOTE :  [0. 2. 0. 0. 0. 0. 1.]
48
NO  :  [9.9975151e-01 5.8268920e-06 7.1667760e-06 2.9293419e-06 2.3401324e-08
 6.2605517e-07 2.3187108e-04]
MID  :  [9.9995732e-01 6.2560622e-07 1.9621295e-06 7.3609683e-09 4.6371058e-09
 1.7814276e-07 4.0010440e-05]
EDGE  :  [7.2440197e-04 3.4025605e-16 1.3794430e-07

69
NO  :  [3.2024136e-10 9.9999988e-01 1.3790889e-10 6.7348068e-08 9.7568575e-09
 3.5448092e-10 1.0263083e-08]
MID  :  [2.5266724e-09 9.9999738e-01 8.6334190e-08 7.3606493e-08 2.2145741e-08
 1.0683933e-07 2.2588690e-06]
EDGE  :  [2.5115730e-03 9.8103702e-01 1.2678246e-06 1.6442491e-02 5.2825626e-06
 8.0274667e-12 2.3396292e-06]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
70
NO  :  [2.1614508e-08 9.9998796e-01 2.0341633e-07 1.1517141e-05 6.7256991e-09
 2.4777127e-09 2.2340998e-07]
MID  :  [2.81195355e-07 9.99816835e-01 7.87047320e-05 3.49614020e-05
 1.29179245e-08 2.22552603e-06 6.70855443e-05]
EDGE  :  [6.1824204e-09 4.1751819e-10 9.7312568e-11 1.0000000e+00 7.1559223e-17
 2.4356924e-17 8.9029689e-10]
VOTE :  [0. 2. 0. 1. 0. 0. 0.]
71
NO  :  [9.9983025e-01 1.8807196e-08 1.3214332e-06 4.6360131e-07 2.8670097e-09
 1.7619276e-07 1.6779618e-04]
MID  :  [9.9999523e-01 1.9860502e-09 3.1181941e-07 6.3447836e-10 1.4299412e-10
 4.9226427e-09 4.4119033e-06]
EDGE  :  [7.9051899e-03 1.4318949e-16 9.9209481e-01

93
NO  :  [1.3702191e-08 9.9995327e-01 8.7410923e-10 4.0627322e-07 1.1115841e-06
 4.3537362e-05 1.7704108e-06]
MID  :  [1.5931947e-09 9.9999607e-01 2.2618384e-10 1.1540019e-08 1.2827770e-07
 2.9841240e-06 8.0053707e-07]
EDGE  :  [5.6603582e-13 1.0000000e+00 4.9547394e-17 6.1464236e-11 4.5750369e-11
 1.8136673e-10 2.1442592e-12]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
94
NO  :  [1.3672231e-11 9.9974185e-01 4.9492042e-12 6.2858385e-09 6.7313870e-08
 2.5752481e-04 5.1126131e-07]
MID  :  [1.45928050e-12 9.99985576e-01 1.14410599e-11 2.09333210e-11
 1.06787024e-09 1.43986435e-05 3.15064277e-08]
EDGE  :  [1.0781574e-14 1.7059194e-05 2.5115310e-07 1.2572345e-06 9.1312354e-15
 3.5398434e-11 9.9998140e-01]
VOTE :  [0. 2. 0. 0. 0. 0. 1.]
95
NO  :  [1.9929120e-04 9.8775685e-01 2.8880882e-05 2.6139501e-03 1.7799326e-03
 3.3107202e-03 4.3104887e-03]
MID  :  [1.2766918e-05 9.9525309e-01 7.9648216e-06 1.5242661e-04 6.0046450e-06
 2.7358954e-04 4.2940429e-03]
EDGE  :  [9.9225396e-01 3.2725943e-06 1.2550681e-08

...

258
NO  :  [6.3438685e-11 9.9999452e-01 1.3300833e-09 1.0149460e-06 4.2074439e-06
 2.8060555e-07 3.6177919e-08]
MID  :  [7.5706216e-13 9.9999559e-01 2.0452987e-06 1.2753844e-07 1.7384169e-06
 3.9676101e-07 7.8750745e-08]
EDGE  :  [1.7670025e-08 9.9998331e-01 9.5515046e-12 3.5234968e-10 1.2653557e-07
 4.2911410e-08 1.6530837e-05]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
259
NO  :  [4.8297824e-13 9.9999988e-01 5.9809219e-10 7.8732576e-09 7.1429312e-08
 2.7818753e-10 8.3930676e-13]
MID  :  [1.6236920e-16 1.0000000e+00 6.0157718e-10 1.4176527e-12 3.0752759e-10
 9.8577240e-12 4.3344625e-13]
EDGE  :  [2.8192739e-14 9.6792054e-01 6.9140077e-10 3.1226175e-06 8.2552617e-14
 1.5018632e-08 3.2076318e-02]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
260
NO  :  [9.9997878e-01 5.0171769e-07 8.9634462e-08 9.0785326e-09 2.9969645e-09
 1.0807732e-07 2.0551353e-05]
MID  :  [9.9999869e-01 3.9420094e-09 2.1419716e-08 1.1318058e-11 2.5627840e-11
 1.2243103e-08 1.2702495e-06]
EDGE  :  [1.8120368e-01 4.9297354e-04 3.8835828e-04 8.0

281
NO  :  [2.7825649e-14 1.0000000e+00 4.2802646e-13 7.0146616e-10 1.4319445e-08
 9.0969250e-09 1.3647261e-10]
MID  :  [1.2388456e-15 1.0000000e+00 1.0543936e-10 6.7402590e-11 1.3841759e-09
 9.5422283e-09 1.0045740e-09]
EDGE  :  [2.3449340e-14 1.0000000e+00 2.4282464e-14 7.0546889e-13 4.7445396e-17
 3.5745184e-15 9.0097819e-15]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
282
NO  :  [4.3678166e-13 1.0000000e+00 9.9597041e-12 6.5673428e-10 3.5109469e-08
 6.7981856e-09 2.1973343e-11]
MID  :  [4.2846526e-16 1.0000000e+00 9.9259524e-11 1.5067767e-12 6.5345528e-11
 1.3043078e-08 1.2194392e-11]
EDGE  :  [4.1703677e-17 6.8975242e-10 7.1502209e-09 1.9273990e-12 1.5133499e-15
 1.3430389e-12 1.0000000e+00]
VOTE :  [0. 2. 0. 0. 0. 0. 1.]
283
NO  :  [1.2942998e-13 1.0000000e+00 1.2863162e-13 7.8595108e-10 5.3379723e-09
 2.8378973e-09 7.8433031e-11]
MID  :  [5.1727153e-16 1.0000000e+00 8.2904143e-13 4.4261465e-12 3.3325948e-10
 1.7370564e-09 1.0181541e-09]
EDGE  :  [1.4476351e-13 1.0000000e+00 1.5020623e-12 9.4

305
NO  :  [3.3372383e-18 7.0723739e-11 2.6111271e-11 9.9999452e-01 1.1923154e-09
 3.3004023e-15 5.5045030e-06]
MID  :  [1.2509144e-16 4.4304407e-11 3.7680640e-11 9.9997163e-01 9.3526567e-12
 4.4033554e-15 2.8312832e-05]
EDGE  :  [2.2064798e-07 6.4741425e-13 6.8463186e-09 9.9991810e-01 3.9508624e-15
 3.6749907e-14 8.1680773e-05]
VOTE :  [0. 0. 0. 3. 0. 0. 0.]
306
NO  :  [3.70114932e-22 1.25006825e-14 5.56374391e-13 9.99999762e-01
 6.97908745e-12 4.78144044e-17 2.13118184e-07]
MID  :  [1.2271925e-24 2.3442325e-15 7.0582817e-11 9.9999881e-01 3.5358761e-15
 4.2102422e-18 1.1343406e-06]
EDGE  :  [1.6640840e-09 8.2359976e-01 6.3862171e-10 1.7571677e-01 4.1636994e-10
 2.5327751e-08 6.8340468e-04]
VOTE :  [0. 1. 0. 2. 0. 0. 0.]
307
NO  :  [3.8716317e-16 2.5951916e-10 1.7617380e-10 9.9999309e-01 3.4653413e-09
 6.8369620e-14 6.9218058e-06]
MID  :  [3.0506842e-13 4.2290549e-09 3.7129146e-09 9.9988258e-01 8.0506896e-10
 1.1082447e-13 1.1735619e-04]
EDGE  :  [5.2300957e-06 2.0539360e-14 4.6246857e

MID  :  [7.9231177e-10 1.4397900e-13 1.0000000e+00 5.5761333e-09 3.6675547e-17
 8.6767110e-19 1.6997467e-08]
EDGE  :  [1.1834704e-04 2.2950253e-11 9.9967194e-01 2.0812747e-04 1.3005392e-23
 1.2285147e-09 1.6694366e-06]
VOTE :  [0. 0. 3. 0. 0. 0. 0.]
329
NO  :  [1.0000000e+00 1.4487684e-12 1.8177106e-08 7.3861368e-12 2.1292328e-12
 1.0008717e-08 5.3792157e-08]
MID  :  [1.0000000e+00 7.4701199e-13 1.3070048e-08 1.1215250e-14 1.5322714e-13
 1.6725432e-11 3.0630838e-09]
EDGE  :  [1.2318929e-08 1.3684593e-07 9.8764938e-01 1.3874400e-07 5.3212457e-16
 4.2504317e-07 1.2349951e-02]
VOTE :  [2. 0. 1. 0. 0. 0. 0.]
330
NO  :  [9.9129311e-06 1.8041153e-08 9.9890375e-01 6.8084431e-05 1.3328204e-09
 1.2450335e-10 1.0182172e-03]
MID  :  [1.8634661e-07 8.4713608e-12 9.9999952e-01 1.7138932e-09 4.3639755e-13
 3.2972711e-16 2.1763158e-07]
EDGE  :  [1.8641149e-06 4.4889670e-16 9.9999809e-01 7.0118553e-13 9.7821462e-20
 1.1619157e-19 4.6323270e-09]
VOTE :  [0. 0. 3. 0. 0. 0. 0.]
331
NO  :  [9.9999571e-01 

352
NO  :  [6.09869360e-12 2.67099473e-14 1.00000000e+00 8.95169702e-11
 1.10743015e-17 1.36078606e-17 3.41794752e-11]
MID  :  [2.6258824e-13 7.4628529e-14 1.0000000e+00 3.2144509e-10 9.5541758e-20
 3.6556505e-19 6.2057504e-10]
EDGE  :  [2.09284390e-05 1.04554074e-16 9.99979019e-01 7.04455799e-11
 6.47944902e-22 2.89973429e-15 1.18800580e-10]
VOTE :  [0. 0. 3. 0. 0. 0. 0.]
353
NO  :  [1.9125255e-14 4.8419919e-18 1.0000000e+00 9.2189589e-11 6.3110299e-18
 9.8323340e-20 1.9565005e-12]
MID  :  [1.0375938e-13 5.6829429e-14 1.0000000e+00 1.8132990e-08 7.1236670e-19
 5.9810180e-20 7.0704065e-10]
EDGE  :  [3.5108462e-07 1.2011785e-15 6.7047559e-02 1.4997484e-08 4.3738819e-21
 5.2687854e-11 9.3295211e-01]
VOTE :  [0. 0. 2. 0. 0. 0. 1.]
354
NO  :  [6.2797458e-12 2.6289780e-14 1.0000000e+00 8.7852059e-11 1.1315577e-17
 1.3485028e-17 3.4422330e-11]
MID  :  [2.7056943e-13 7.2451891e-14 1.0000000e+00 3.1089162e-10 9.7891455e-20
 3.6100163e-19 6.0859989e-10]
EDGE  :  [2.9472063e-05 1.3975453e-16 9.9

# 網站爬圖 by BeautifulSoup (沒啥用)

In [12]:
import requests
from bs4 import BeautifulSoup
import os

def getHtmlurl(url): #獲取網址
    try:
        r=requests.get(url)
        r.raise_for_status()
        r.encoding=r.apparent_encoding
        return r.text
    except:
        return ""

def getpic(html): #獲取圖片地址並下載
    soup =BeautifulSoup(html,'html.parser')
    all_img=soup.find_all('img')
    for img in all_img:
        src=img['src']
        img_url=src
        print (img_url)
        root='data_extra/NET2/'
        path = root + img_url.split('/')[-1]
        print (path)
        try: #創建或判斷路徑圖片是否存在並下載
            if not os.path.exists(root):
                os.mkdir(root)
            if not os.path.exists(path):
                r = requests.get(img_url)
                with open(path, 'wb') as f:
                    f.write(r.content)
                    f.close()
                    print("文件保存成功")
            else:
                print("文件已存在")
        except:
            print("爬取失敗")



def main():
    url='http://asamai.weebly.com/'
    html=(getHtmlurl(url))

    print(getpic(html))
    
main()

http://www.f-counter.net/j/22/1420026157/
data_extra/NET2/
文件已存在
/uploads/2/3/2/0/23202642/1470581952208.jpg?193
data_extra/NET2/1470581952208.jpg?193
爬取失敗
//cdn2.editmysite.com/images/site/footer/footer-toast-published-image-1.png
data_extra/NET2/footer-toast-published-image-1.png
爬取失敗
None


# 網站爬圖 by Icrawler

In [85]:
from icrawler.builtin import BaiduImageCrawler
from icrawler.builtin import BingImageCrawler
from icrawler.builtin import GoogleImageCrawler

"""
parser_threads : 解析器线程数目，最大为cpu数目
downloader_threads : 下载线程数目，最大为cpu数目
storage : 存储地址，使用字典格式。key为root_dir
keyword : 浏览器搜索框输入的关键词
max_num : 最大下载图片数目
"""

import time
import os

KEY_WORD   = '五更琉璃'
KEY_LABEL  = '5'
IMAGE_NUM  = 50
BROWSER_ID = [1, 0, 0]  # 0:Google  1:Bing  2:Baidu

LABELS = [
    '0_Izumi Sagiri_和泉紗霧',
    '1_Izumi Masamune_和泉正宗',
    '2_Yamada Erufu_山田妖精',
    '3_Jinno Megumi_神野惠',
    '4_Muramasa Senju_千壽村征',
    '5_Takasago Tomoe_高砂智惠',
    '6_Kagurazaka Ayame_神樂坂菖蒲',
]

TIME = time.strftime ("_%Y-%m-%d_%H'%M'%S", time.localtime())


def Crawler_Recognition (IP, OP, L, DV):
    
    dIMAGES = os.listdir(IP)
    
    if not os.path.exists(OP):
        os.mkdir(OP)
    
    imageT_i = 0
    imageF_i = 0
    for dIMAGE in dIMAGES:
        
        fname, ftype = os.path.splitext(dIMAGE)
        ftype = '.jpg'
        
        faces = []
        for (a,b,dw,dh) in DV:
            img, faces_ = Get_Image_Animeface_Detection (IP + '/' + dIMAGE, None, a, b, dw, dh)
            faces = faces + faces_
        
        img_show   = cv2.imread(IP + '/' + dIMAGE)
        
        img_T = 0
        for (x, y, w, h) in faces:
            
            cv2.rectangle(img_show,(x,y),(x+w,y+h),(255,0,0),5)
            
            ans_str = Get_Image_Animeface_Recognition (cv2.resize(img[y:y+h, x:x+w], (INPUT_SIZE, INPUT_SIZE)), CROP_LIST, PLAN)
            print('ans : ', ans_str)
            
            if (ans_str == L):
                img_T = 1
                cv2.rectangle(img_show,(x,y),(x+w,y+h),(0,0,255),5)
                break
        
        os.remove  (IP + '/' + dIMAGE)
        if(img_T == 1):
            print(OP + '/' + str(imageT_i) + ftype)
            cv2.imwrite(OP + '/' + str(imageT_i) + ftype, img_show)
            imageT_i += 1
        elif img_show is not None:
            print(IP + '/' + str(imageF_i) + ftype)
            cv2.imwrite(IP + '/' + str(imageF_i) + ftype, img_show)
            imageF_i += 1
            
        print("-----------------------------------------------------")

# Google
if (BROWSER_ID[0] == 1):
    PATH = 'data_extra/Google'
    google_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    google_crawler = GoogleImageCrawler(parser_threads=2, downloader_threads=4, storage=google_storage)
    google_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL, DETECT_VAR_1)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)

# Bing
if (BROWSER_ID[1] == 1):
    PATH = 'data_extra/Bing'
    bing_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    bing_crawler = BingImageCrawler(parser_threads=2,  downloader_threads=4, storage=bing_storage)
    bing_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)

# Baidu
if (BROWSER_ID[2] == 1):
    PATH = 'data_extra/Baidu'
    baidu_storage = {'root_dir':'/Users/user/Desktop/jupyter/PatternProject/' + PATH + '/dealing'}
    baidu_crawler = BaiduImageCrawler(parser_threads=2, downloader_threads=4, storage=baidu_storage)
    baidu_crawler.crawl(keyword=KEY_WORD, max_num=IMAGE_NUM)
    
    Crawler_Recognition (PATH + '/dealing', PATH + '/dealing/dealing', KEY_LABEL)
    os.rename(PATH + '/dealing/dealing', PATH + '/dealing/' + LABELS[int(KEY_LABEL)])
    os.rename(PATH + '/dealing'        , PATH + '/' + KEY_WORD + TIME)


2019-01-01 04:43:29,970 - INFO - icrawler.crawler - start crawling...
2019-01-01 04:43:29,975 - INFO - icrawler.crawler - starting 1 feeder threads...
2019-01-01 04:43:29,993 - INFO - feeder - thread feeder-001 exit
2019-01-01 04:43:30,002 - INFO - icrawler.crawler - starting 2 parser threads...
2019-01-01 04:43:30,032 - INFO - icrawler.crawler - starting 4 downloader threads...
2019-01-01 04:43:30,327 - INFO - parser - parsing result page https://www.google.com/search?ijn=0&tbm=isch&start=0&tbs=&q=%E4%BA%94%E6%9B%B4%E7%90%89%E7%92%83
2019-01-01 04:43:30,868 - INFO - downloader - image #1	https://vignette.wikia.nocookie.net/oreimo/images/f/f0/Kuroneko.jpg/revision/latest?cb=20110602001044&path-prefix=zh
2019-01-01 04:43:31,050 - INFO - downloader - image #2	http://a4.att.hudong.com/81/75/01300001128119143132751735362_s.jpg
2019-01-01 04:43:31,108 - INFO - downloader - image #3	https://gss2.bdstatic.com/9fo3dSag_xI4khGkpoWK1HF6hhy/baike/c0%3Dbaike60%2C5%2C5%2C60%2C20/sign=c698b6ebf9edab

2019-01-01 04:43:42,363 - INFO - downloader - downloaded images reach max num, thread downloader-001 is ready to exit
2019-01-01 04:43:42,365 - INFO - downloader - thread downloader-001 exit
2019-01-01 04:43:42,426 - INFO - icrawler.crawler - Crawling task done!


NO  :  [3.1776931e-10 2.4236625e-02 6.9687846e-11 4.0490173e-07 7.0816453e-04
 9.7505379e-01 1.0477422e-06]
MID  :  [6.2391052e-12 7.5037628e-02 1.7644511e-13 3.1513520e-10 1.9460726e-04
 9.2476583e-01 2.0350617e-06]
EDGE  :  [1.1546684e-06 1.6634083e-11 9.9915481e-01 1.8935025e-09 1.0599211e-15
 1.5890770e-11 8.4395666e-04]
VOTE :  [0. 0. 1. 0. 0. 2. 0.]
ans :  5
data_extra/Google/dealing/dealing/0.jpg
-----------------------------------------------------
NO  :  [8.2139996e-11 3.7194660e-01 1.0900103e-10 8.8691792e-07 6.2700093e-01
 1.0503659e-03 1.1151529e-06]
MID  :  [2.37114529e-11 8.27574193e-01 5.04107242e-11 1.22507355e-08
 1.71676323e-01 6.27125904e-04 1.22340876e-04]
EDGE  :  [3.3483952e-11 9.9840122e-01 2.3893976e-10 4.0054890e-10 6.9024382e-09
 2.9095723e-05 1.5697253e-03]
VOTE :  [0. 2. 0. 0. 1. 0. 0.]
ans :  1
NO  :  [6.12324903e-15 1.77777687e-03 2.13173815e-12 4.42149206e-09
 9.96239185e-01 1.98305305e-03 1.32036115e-08]
MID  :  [2.1972164e-15 3.6049483e-03 1.2441984e-13

data_extra/Google/dealing/4.jpg
-----------------------------------------------------
NO  :  [2.3242005e-03 1.0556245e-03 3.3081502e-05 8.1989936e-07 4.9012020e-01
 2.3834714e-01 2.6811895e-01]
MID  :  [5.6972372e-04 9.4575427e-02 7.4676592e-08 2.8638341e-09 6.6928923e-01
 5.8741201e-02 1.7682435e-01]
EDGE  :  [1.23957165e-08 1.41185626e-06 2.84553080e-05 4.52687374e-07
 2.62407185e-09 1.48259933e-05 9.99954820e-01]
VOTE :  [0. 0. 0. 0. 1. 0. 1.]
ans :  unknow
NO  :  [2.2161547e-02 1.5009840e-03 9.2984199e-05 3.8670640e-07 1.5637361e-02
 1.8544463e-04 9.6042132e-01]
MID  :  [8.01095545e-01 3.64214880e-04 1.77778165e-05 3.86755588e-08
 1.04112096e-01 1.67416816e-03 9.27361026e-02]
EDGE  :  [2.5912288e-05 4.7857654e-13 1.3752697e-02 1.4564690e-10 9.1431508e-22
 2.2876851e-16 9.8622143e-01]
VOTE :  [1. 0. 0. 0. 0. 0. 2.]
ans :  6
data_extra/Google/dealing/5.jpg
-----------------------------------------------------
NO  :  [1.1648823e-12 1.3639186e-04 6.8597100e-11 8.0725373e-09 8.2873303e-

NO  :  [1.01313745e-07 3.40225088e-04 2.14539897e-09 2.32942412e-07
 6.69910759e-02 9.32388484e-01 2.79831642e-04]
MID  :  [1.3686752e-10 1.4983752e-06 2.3624139e-11 4.8910109e-10 1.3849945e-01
 8.5997957e-01 1.5193719e-03]
EDGE  :  [3.09949764e-03 2.89402262e-04 6.38866113e-05 1.02954125e-08
 2.47088822e-10 4.78538610e-02 9.48693275e-01]
VOTE :  [0. 0. 0. 0. 0. 2. 1.]
ans :  5
data_extra/Google/dealing/dealing/18.jpg
-----------------------------------------------------
NO  :  [4.64686991e-13 9.96360719e-01 2.89344604e-10 3.21926308e-07
 1.28555985e-05 3.61492718e-03 1.11389481e-05]
MID  :  [1.5930828e-13 9.7278023e-01 5.0801266e-11 1.8941688e-09 1.2926193e-05
 2.7159737e-02 4.7149129e-05]
EDGE  :  [1.4805629e-07 7.5268412e-01 4.5833740e-08 7.7591458e-04 2.7682833e-07
 4.8200562e-08 2.4653932e-01]
VOTE :  [0. 3. 0. 0. 0. 0. 0.]
ans :  1
NO  :  [2.8112776e-11 5.3433675e-01 8.9684551e-07 6.8213349e-06 1.6434676e-03
 4.6355939e-01 4.5260877e-04]
MID  :  [6.0548541e-12 1.8582085e-02 1.670

 7.1701002e-01 1.5793669e-01]
EDGE  :  [1.4055538e-09 1.7834983e-06 2.6084976e-02 4.9059502e-16 1.1519185e-30
 9.8076427e-19 9.7391325e-01]
VOTE :  [1. 0. 0. 0. 0. 1. 1.]
ans :  unknow
NO  :  [6.0568092e-04 1.8391305e-03 7.3677520e-06 5.2220011e-01 4.5716473e-01
 3.4673247e-03 1.4715619e-02]
MID  :  [4.8672728e-09 2.7706557e-05 2.5016098e-08 4.5526256e-05 9.6045113e-01
 1.1766710e-03 3.8298935e-02]
EDGE  :  [8.0476385e-03 9.1616797e-13 1.2530126e-01 1.4496446e-04 2.5251370e-20
 6.2122502e-10 8.6650616e-01]
VOTE :  [0. 0. 0. 1. 1. 0. 1.]
ans :  unknow
NO  :  [7.3975286e-08 3.1769142e-04 5.1426237e-08 2.7484539e-06 9.9954802e-01
 1.3114524e-04 2.0802993e-07]
MID  :  [6.2294087e-11 7.8681396e-06 1.1845581e-10 1.6972660e-08 9.9987459e-01
 1.1645808e-04 1.0287048e-06]
EDGE  :  [2.21485243e-05 9.89799857e-01 1.29661849e-03 1.00350015e-07
 1.60194933e-04 7.86400307e-03 8.57007632e-04]
VOTE :  [0. 1. 0. 0. 2. 0. 0.]
ans :  4
NO  :  [3.6803051e-06 2.0637160e-06 2.0651866e-08 1.8920311e-07 1.239

-----------------------------------------------------


# Free Coding

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import time
 
# 格式化成2016-03-20 11:45:39形式
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

In [15]:
import numpy as np

PPP = 10

def AA ():
    global PPP
    
    PPP = 111

AA()
print('ans', PPP)

ans 111


In [410]:
a = [[1,2], [3,4]]
b = [[3,0], [0,5]]
print(a)
print(b)
print(a+b)

[[1, 2], [3, 4]]
[[3, 0], [0, 5]]
[[1, 2], [3, 4], [3, 0], [0, 5]]


In [41]:
IP = 'data'

for dCHARACTER_DIR_NAME in os.listdir(IP):
    
    label, name = dCHARACTER_DIR_NAME.split('_', 1)

    IP_d = IP + '/' + dCHARACTER_DIR_NAME
    for dEPISODE in os.listdir(IP_d):
        
        IP_dd = IP_d + '/' + dEPISODE
        os.rename(IP_dd, IP_dd + '.jpg')
        """
        IP_dd = IP_d + '/' + dEPISODE
        for dIMAGE in os.listdir(IP_dd):
            
            a, b, c, d, e = dIMAGE.split('_', 4)
            e1, e2 = os.path.splitext(e)
            mark = a+'_'+b+'_'+c+'_'+d+e1
            print(mark)
            os.rename(IP_dd + '/' + dIMAGE, IP_d + '/' + mark)
        """